Import statements

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [4]:
nltk.download('punkt')
nltk.download('all')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /r

True

Compile and run from here.

To load the train and validation data from TSV files.

Arguments: train file, validation file

Returns:

tr_st: Training sentences
tr_ft: Training Future Labels
tr_pt: Trainiing past labels
val_st: Validation sentences
val_ft: Validation Future Labels
val_pt: Validation past labels

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


Method to load the test data from TSV file.

Arguments: test file

Returns:

ts_st: Test sentences
ts_ft: Test Future Labels
ts_pt: Test past labels

In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

To preprocess sentences:
Removing some special characters, spaces and tabs, and conversion to lower case

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

Converts sentences to lists of features

Uncomment code as needed for features



In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word+' '+pos for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
#     document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs


This is not necesasry unless the code is being run on a Colab notebook, and the data is stored on Google Drive

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Path to the location where the data resides

In [0]:
datapath = 'gdrive/My Drive/data/'

Loading the train and validation data

Preprocessing the data

Feature extraction if necessary

In [16]:
#loading train and validation data
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [17]:
print(tr_st[1])

friends NNS and CC adversaries NNS abroad RB were VBD asking VBG whether IN america NN had VBD lost VBN its PRP$ nerve NN


Imports necessary for LSTM 

In [0]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras import optimizers

# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import numpy as np
import pandas as pd
import re
from tqdm import tqdm


Creating a Keras Tokenizer and fitting it on train and validation data

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

In [0]:
from gensim.models import Word2Vec

pos_model = Word2Vec.load(datapath+'model_300d.bin')
# print(pos_model)
words = list(pos_model.wv.vocab)
# print(words)
# print(len(words))
# print(pos_model['NN'])



Initializing vocabulary size

In [0]:
vocabulary_size = len(tokenizer.word_index)+1
# print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_docs)


In [22]:
print(vocabulary_size)

6771


Extracting sequences from train and validation data, and padding them to a maximum length

In [32]:

# vocabulary_size = 7000
# tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))
MAX_LEN=145+145
sequences = tokenizer.texts_to_sequences(tr_st)
tr_data = pad_sequences(sequences, maxlen=MAX_LEN)
sequences = tokenizer.texts_to_sequences(val_st)
val_data = pad_sequences(sequences, maxlen=MAX_LEN)

print(len(tr_data))
print(len(val_data))

1776
592


Loading Glove embeddings

In [24]:
f = open(datapath+'glove.6B.300d.txt')
embedding_values = {}
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float32')
  embedding_values[word]=coef

400000it [00:37, 10784.79it/s]


Adding embedding values of POS tags

In [26]:
for word in words:
  coef=np.array(pos_model[word],dtype = 'float32')
  embedding_values[word]=coef

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [27]:
len(embedding_values)

400032

Creating an embedding matrix for words 

In [29]:
embedding_matrix = np.zeros((vocabulary_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  values = embedding_values.get(word)
  if values is not None:
    embedding_matrix[i] = values

100%|██████████| 6770/6770 [00:00<00:00, 269536.85it/s]


In [0]:
import numpy as np
import tensorflow as tf
import random as rn
from numpy.random import seed
from tensorflow import set_random_seed


In [0]:
def session_refresh():
  seed(2)
  set_random_seed(2)
  rn.seed(2)
  os.environ['PYTHONHASHSEED'] = '0'
  session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
  K.set_session(sess)
  print('refreshed session')

BiLSTM for Future task

Stacked BiLSTM for Future task

In [37]:
#Future LSTM2
# K.clear_session()
K.set_session(sess)
model2 = Sequential()
print(MAX_LEN)
model2.add(Embedding(vocabulary_size,300, input_length = MAX_LEN,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)

filepath=datapath+'/models/'+"future-2-pos-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


290
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 290, 300)          2031300   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 290, 290)          517360    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 290)               505760    
_________________________________________________________________
dense_5 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 3,073,109
Trainable params: 1,041,809
Non-trainable params: 2,031,300
_________________________________________________________________


In [35]:
model2.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=10,callbacks=callbacks_list,batch_size=64)  #epoch 15 batch 128 no relu hidden   epoch 7 batch 128 w/ relu hidden
model2_json = model2.to_json()
with open(datapath+"future-2-pos-model.json", "w") as json_file:
    json_file.write(model2_json)

Instructions for updating:
Use tf.cast instead.
Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 54s 31ms/step - loss: 0.7051 - acc: 0.5755 - val_loss: 1.1178 - val_acc: 0.4324

Epoch 00001: val_acc improved from -inf to 0.43243, saving model to gdrive/My Drive/data//models/future-2-pos-best.hdf5
Epoch 2/10
1776/1776 [==============================] - 51s 29ms/step - loss: 0.6604 - acc: 0.6875 - val_loss: 0.5319 - val_acc: 0.7584

Epoch 00002: val_acc improved from 0.43243 to 0.75845, saving model to gdrive/My Drive/data//models/future-2-pos-best.hdf5
Epoch 3/10
1776/1776 [==============================] - 51s 29ms/step - loss: 0.5335 - acc: 0.7573 - val_loss: 0.5221 - val_acc: 0.7652

Epoch 00003: val_acc improved from 0.75845 to 0.76520, saving model to gdrive/My Drive/data//models/future-2-pos-best.hdf5
Epoch 4/10
1776/1776 [==============================] - 50s 28ms/step - loss: 0.5003 - acc: 0.7652 - val_loss: 0.5096 - val_acc:

NameError: ignored

Training on whole training data after parameter tuning, in order to use the model to evaluate the test data

In [38]:
# model2.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=9,batch_size=64)

# model_json = model2.to_json()
# with open(datapath+'/models/'+"future-2-full-pos-model.json", "w") as json_file:
#     json_file.write(model_json)
# model2.save_weights(datapath+'/models/'+"future-2-full-pos-best.hdf5")
# print("Saved model to disk")

Epoch 1/9
2368/2368 [==============================] - 62s 26ms/step - loss: 0.6540 - acc: 0.6423
Epoch 2/9
2368/2368 [==============================] - 60s 25ms/step - loss: 0.5397 - acc: 0.7580
Epoch 3/9
2368/2368 [==============================] - 60s 25ms/step - loss: 0.5438 - acc: 0.7416
Epoch 4/9
2368/2368 [==============================] - 61s 26ms/step - loss: 0.4909 - acc: 0.7821
Epoch 5/9
2368/2368 [==============================] - 59s 25ms/step - loss: 0.4750 - acc: 0.7927
Epoch 6/9
2368/2368 [==============================] - 60s 25ms/step - loss: 0.4659 - acc: 0.7893
Epoch 7/9
2368/2368 [==============================] - 60s 25ms/step - loss: 0.4415 - acc: 0.8015
Epoch 8/9
2368/2368 [==============================] - 59s 25ms/step - loss: 0.4210 - acc: 0.8155
Epoch 9/9
2368/2368 [==============================] - 61s 26ms/step - loss: 0.4034 - acc: 0.8298
Saved model to disk


Loading the Test data

preprocessing the data

Extracting sequences

Padding the sequences



In [39]:
testfile=datapath+'test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
#ts_st = sent_to_features(ts_st)
sequences = tokenizer.texts_to_sequences(ts_st)
ts_data = pad_sequences(sequences, maxlen=MAX_LEN)
print(ts_data.shape)
print(ts_ft.shape)
# test_loss, test_acc = model.evaluate(np.append((ts_data,ts_ft),1))
# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))


# print('Accuaracy = {0:f}'.format(acc))

593
(593, 290)
(593,)


In [42]:
from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"future-2-pos-model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights(datapath+'/models/'+"future-2-pos-best.hdf5")
print("Loaded model from disk")
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

Loaded model from disk


Evaluating the test data for future task 

In [43]:

scores = model2.evaluate(ts_data, ts_ft, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 76.22%


Future Test Accuracy: 82.80% ; epochs 5+3, model: BiLSTM + Dense

Accuracy: 72.68%



Generating a classification report for the test data for future task

In [44]:
from sklearn.metrics import classification_report
y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

593/593 [==============================] - 7s 11ms/step
              precision    recall  f1-score   support

         0.0       0.77      0.74      0.75       293
         1.0       0.76      0.78      0.77       300

   micro avg       0.76      0.76      0.76       593
   macro avg       0.76      0.76      0.76       593
weighted avg       0.76      0.76      0.76       593





Future Test Accuracy: 82.80% ; epochs 5+3, model: BiLSTM + Dense

precision    recall  f1-score   support

         0.0       0.87      0.77      0.82       293
         1.0       0.80      0.88      0.84       300

   micro avg       0.83      0.83      0.83       593
   macro avg       0.83      0.83      0.83       593
weighted avg       0.83      0.83      0.83       593




              precision    recall  f1-score   support

         0.0       0.71      0.74      0.73       293
         1.0       0.74      0.71      0.72       300

   micro avg       0.73      0.73      0.73       593
   macro avg       0.73      0.73      0.73       593
weighted avg       0.73      0.73      0.73       593


BiLSTM model for Past task

Stacked BiLSTM for Past task

In [47]:
#Past LSTM2
session_refresh()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,300, input_length = MAX_LEN,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath=datapath+'/models/'+"past-2-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model2.summary()


refreshed session
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 290, 300)          2031300   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 290, 290)          517360    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 290)               505760    
_________________________________________________________________
dense_9 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 3,073,109
Trainable params: 1,041,809
Non-trainable params: 2,031,300
_________________________________________________________________


In [48]:
model2.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=10,callbacks=callbacks_list,batch_size=64) #eposh 8 batch 128 no relu hidden
model2_json = model2.to_json()
with open(datapath+"past-2-model.json", "w") as json_file:
    json_file.write(model2_json)

Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 54s 31ms/step - loss: 0.6876 - acc: 0.6053 - val_loss: 0.6041 - val_acc: 0.6740

Epoch 00001: val_acc improved from -inf to 0.67399, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 2/10
1776/1776 [==============================] - 48s 27ms/step - loss: 0.6368 - acc: 0.6678 - val_loss: 0.5584 - val_acc: 0.7416

Epoch 00002: val_acc improved from 0.67399 to 0.74155, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 3/10
1776/1776 [==============================] - 50s 28ms/step - loss: 0.5412 - acc: 0.7545 - val_loss: 0.5322 - val_acc: 0.7568

Epoch 00003: val_acc improved from 0.74155 to 0.75676, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 4/10
1776/1776 [==============================] - 51s 29ms/step - loss: 0.4852 - acc: 0.8018 - val_loss: 0.5492 - val_acc: 0.7635

Epoch 00004: val_acc improved from 0.75676 to 0.76351, sa

In [0]:
# model2.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_pt,val_pt)), epochs=9,batch_size=64)
# model2_json = model2.to_json()
# with open(datapath+"past-2-full-model.json", "w") as json_file:
#     json_file.write(model2_json)
# model2.save_weights(datapath+'/models/'+"past-2-full-best.hdf5")
# print("Saved model to disk")

Epoch 1/1
2368/2368 [==============================] - 37s 15ms/step - loss: 0.3353 - acc: 0.8611
Saved model to disk


In [49]:
from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"past-2-model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights(datapath+'/models/'+"past-2-best.hdf5")
print("Loaded model from disk")
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

Loaded model from disk


Evaluating the test data for future task 

In [52]:
scores = model2.evaluate(ts_data, ts_pt, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))



Accuracy: 72.34%


Accuracy: 78.75%


w/pos: Accuracy: 72.34%

In [0]:
from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"past-2-full-model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights(datapath+'/models/'+"past-2-full-best.hdf5")
print("Loaded model from disk")
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

Loaded model from disk


Generating a classification report for the test data for future task

In [51]:
from sklearn.metrics import classification_report
y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_pt, y_pred))

593/593 [==============================] - 7s 12ms/step
              precision    recall  f1-score   support

         0.0       0.67      0.96      0.79       324
         1.0       0.90      0.44      0.59       269

   micro avg       0.72      0.72      0.72       593
   macro avg       0.79      0.70      0.69       593
weighted avg       0.78      0.72      0.70       593



Past Test Accuracy: 78.75% ; epochs 9, model: BiLSTM + Dense

precision    recall  f1-score   support

         0.0       0.78      0.85      0.81       324
         1.0       0.80      0.71      0.75       269

   micro avg       0.79      0.79      0.79       593
   
   macro avg       0.79      0.78      0.78       593
   
weighted avg       0.79      0.79      0.79       593
